Algorytm Symulowanego Wyżarzania SA
Paweł Brodziak
Magdalena Leśniak
Marceli Ptak

In [30]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import random
import mpmath
import math
import time
import warnings
from enum import Enum
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
TSP_29 = pd.read_excel('TSP_29.xlsx')
TSP_29.name = 'TSP_29'
TSP_76 = pd.read_excel('TSP_76.xlsx')
TSP_76.name = 'TSP_76'
TSP_127 = pd.read_excel('TSP_127.xlsx')
TSP_127.name = 'TSP_127'
for df in [TSP_29, TSP_76, TSP_127]:
    df.index = np.arange(1, len(df) + 1)

In [36]:
class KryteriumStopu(Enum):
    iteracje = 'maksymalna liczba iteracji'
    max_bez_poprawy = 'maksymalna liczba iteracji bez poprawy'

class RedukcjaTemperatury(Enum):
    szybka = 'szybkie wyżarzanie - temp = temp0/(iteracje+1)'
    stopniowa = 'stopniowa redukcja temperatury - temp = temp0 * (1 - iteracje/max_iteracje) - tylko z kryterium stopu max iteracje'
    logarytmicza = 'logarytmiczna redukcja temperatur temp = temp0 * log(iteracje+1)'

##Oblicz odleglości między miastami i całkowitą odległość dla zadanej kolejności
def zmierz_odleglosci(miasta,df):
    odleglosci = []
    for i in range(len(miasta)-1):
        x = df[miasta[i]].iloc[miasta[i+1]-1]
        odleglosci.append(x)
        i+=1
    odleglosci.append(df[miasta[len(df)-1]].iloc[miasta[0]-1])
    suma_odleglosci = sum(odleglosci)
    return odleglosci, suma_odleglosci

## Losowo zamienia kolejność 2 miast z listy kolejności
def znajdz_sasiedztwo(miasta, suma_odleglosci, df):
    ob_odleglosc = sys.maxsize
    nowe_miasta = []
    for i in range(len(miasta)):
        for j in range(i+1,len(miasta)):
            if i == j :
                continue
            temp = np.copy(miasta)
            temp[i], temp[j] = temp[j], temp[i].copy()
            temp_odleglosci, temp_odleglosc = zmierz_odleglosci(temp,df)
            if temp_odleglosc < ob_odleglosc:
                nowe_miasta = temp
                ob_odleglosc = temp_odleglosc
    return nowe_miasta

def wyzarzanie(df, kryterium_stopu:KryteriumStopu, redukcja:RedukcjaTemperatury, temperatura_poczatkowa=100, max_iteracje=10000, max_iteracje_bez_poprawy=1000):
    miasta = [i for i in range(1,len(df)+1)]
    random.shuffle(miasta) ##losowa kolejność miast - wartość początkowa

    odleglosci, suma_odleglosci = zmierz_odleglosci(miasta,df)

    miasta_poczatkowe = miasta
    wartosc_poczatkowa = suma_odleglosci

    iteracje_bez_poprawy = 0
    iteracje_z_poprawa = 0
    temperatura = temperatura_poczatkowa
    historia_odleglosci = []
    iteracje = 0
    climb = True

    while climb:
        if redukcja == RedukcjaTemperatury.szybka:
            temperatura = temperatura_poczatkowa/float(iteracje+1)
        if redukcja == RedukcjaTemperatury.stopniowa:
            temperatura = temperatura_poczatkowa * (1-(iteracje/max_iteracje))
        if redukcja == RedukcjaTemperatury.logarytmicza:
            temperatura =  math.log(temperatura_poczatkowa,iteracje+2)

        miasta_alt = znajdz_sasiedztwo(miasta,suma_odleglosci,df)
        odleglosci_alt, suma_odleglosci_alt = zmierz_odleglosci(miasta_alt,df)
        roznica = suma_odleglosci_alt - suma_odleglosci
        funkcja_metropolis = math.exp(-roznica/temperatura)
        #print(f'nowe odleglosc: {suma_odleglosci_alt}, różnica: {roznica}, metropolis: {funkcja_metropolis}')
        if roznica < 0 or funkcja_metropolis > random.random():
            miasta = miasta_alt
            odleglosci = odleglosci_alt
            suma_odleglosci = suma_odleglosci_alt
            iteracje_bez_poprawy = 0
            iteracje_z_poprawa += 1
            historia_odleglosci.append(suma_odleglosci)
        else:
            iteracje_bez_poprawy +=1
            pass

        iteracje += 1

        if kryterium_stopu == KryteriumStopu.iteracje and iteracje == max_iteracje:
            climb = False
        if kryterium_stopu == KryteriumStopu.max_bez_poprawy and iteracje_bez_poprawy >= max_iteracje_bez_poprawy:
            climb = False

    return miasta, odleglosci, suma_odleglosci, iteracje, iteracje_z_poprawa, temperatura, miasta_poczatkowe, wartosc_poczatkowa, historia_odleglosci

def wyzarzanie_podsumowanie(df, kryterium_stopu:KryteriumStopu = KryteriumStopu.iteracje, redukcja:RedukcjaTemperatury = RedukcjaTemperatury.stopniowa, temperatura_poczatkowa=100, max_iteracje=10000, max_iteracje_bez_poprawy=1000):
    start_time = time.time()
    miasta, odleglosci, suma_odleglosci, iteracje, iteracje_poprawne, temperatura, miasta_poczatkowe, wartosc_poczatkowa, historia_odleglosci = wyzarzanie(df, kryterium_stopu, redukcja, temperatura_poczatkowa, max_iteracje,max_iteracje_bez_poprawy)
    end_time = time.time()
    execution_time = end_time - start_time
    print(f'''
    Kryterium stopu: {kryterium_stopu.value}
    liczba iteracji: {iteracje}
    liczba iteracji z poprawą: {iteracje_poprawne}
    Czas obliczeń: {execution_time:.2f} sekund
    wynik - suma odległości: {suma_odleglosci}
    kolejność miast: {miasta}
    Funkcja redukcji temperatury: {redukcja.value}
    temperatura początkowa: {temperatura_poczatkowa}
    temperatura końcowa {temperatura}
    Wylosowane miasta początkowe: {miasta_poczatkowe}
    Dały wartość początkową (pierwsza suma odległości): {wartosc_poczatkowa}
    W sprawdzanych możliwościach najniższym osiągniętym wynikiem był: {min(historia_odleglosci)}
    ''')

In [38]:
wyzarzanie_podsumowanie(
    TSP_29,
    kryterium_stopu=KryteriumStopu.max_bez_poprawy,
    redukcja=RedukcjaTemperatury.szybka,
    temperatura_poczatkowa=10000,
    max_iteracje_bez_poprawy=10
)


    Kryterium stopu: maksymalna liczba iteracji bez poprawy
    liczba iteracji: 4747
    liczba iteracji z poprawą: 3649
    Czas obliczeń: 320.44 sekund
    wynik - suma odległości: 2303
    kolejność miast: [25 11 22 14 17 18 13 21  1 28  8 27 24 16 19 15  4 10 20  2  3 29 26  5
  9 12  6 23  7]
    Funkcja redukcji temperatury: szybkie wyżarzanie - temp = temp0/(iteracje+1)
    temperatura początkowa: 10000
    temperatura końcowa 2.106593638087213
    Wylosowane miasta początkowe: [3, 18, 23, 11, 16, 8, 14, 17, 6, 1, 4, 5, 27, 15, 29, 2, 28, 22, 19, 20, 24, 9, 25, 10, 21, 12, 26, 13, 7]
    Dały wartość początkową (pierwsza suma odległości): 6175
    W sprawdzanych możliwościach najniższym osiągniętym wynikiem był: 2303
    


In [41]:
wyzarzanie_podsumowanie(
    TSP_29,
    kryterium_stopu=KryteriumStopu.iteracje,
    redukcja=RedukcjaTemperatury.logarytmicza,
    temperatura_poczatkowa=1000,
    max_iteracje=5000
)


    Kryterium stopu: maksymalna liczba iteracji
    liczba iteracji: 5000
    liczba iteracji z poprawą: 2544
    Czas obliczeń: 704.91 sekund
    wynik - suma odległości: 2774
    kolejność miast: [25 19 13 21  2 20 10  4 16 24 27 23  8  1 26  3 29 18 17 14 22 11 15  5
  9 12  6 28  7]
    Funkcja redukcji temperatury: logarytmiczna redukcja temperatur temp = temp0 * log(iteracje+1)
    temperatura początkowa: 1000
    temperatura końcowa 0.8110175425694691
    Wylosowane miasta początkowe: [25, 19, 10, 23, 12, 8, 4, 20, 15, 18, 21, 27, 13, 1, 2, 22, 29, 17, 6, 14, 24, 11, 16, 9, 3, 26, 5, 28, 7]
    Dały wartość początkową (pierwsza suma odległości): 5473
    W sprawdzanych możliwościach najniższym osiągniętym wynikiem był: 2774
    


In [42]:
wyzarzanie_podsumowanie(
    TSP_127,
    kryterium_stopu=KryteriumStopu.iteracje,
    redukcja=RedukcjaTemperatury.stopniowa,
    temperatura_poczatkowa=2000,
    max_iteracje=2000
)


    Kryterium stopu: maksymalna liczba iteracji
    liczba iteracji: 2000
    liczba iteracji z poprawą: 2000
    Czas obliczeń: 18620.30 sekund
    wynik - suma odległości: 186636.6898663332
    kolejność miast: [ 53 118  48  54  35  37  36  40  44 103  45  94  46 112 111 107   2   1
  72  73  69 119  63 102 101  83 117  78   9  11  61  91 125  89  92 104
  85  86  88  87 110  71  20  14  43  34  39  42  28 122  29  32  76 109
  96  82 126  81  84  75  70  68  74  77  18  21  19   4 108 106   7 121
   5  50  24  23   8  67  59  99  65 113  62  60 116  90   3  10 120 114
   6  22  17  79  80  16 124  55  66  64  58 100  93 127  98  97 123  95
  57  56  52 115  13 105  15  12  31  27  26  25  33  38  30  41  51  47
  49]
    Funkcja redukcji temperatury: stopniowa redukcja temperatury - temp = temp0 * (1 - iteracje/max_iteracje) - tylko z kryterium stopu max iteracje
    temperatura początkowa: 2000
    temperatura końcowa 0.9999999999998899
    Wylosowane miasta początkowe: [47, 126, 

In [43]:
wyzarzanie_podsumowanie(
    TSP_76,
    kryterium_stopu=KryteriumStopu.max_bez_poprawy,
    redukcja=RedukcjaTemperatury.szybka,
    temperatura_poczatkowa=10000,
    max_iteracje_bez_poprawy=20
)


    Kryterium stopu: maksymalna liczba iteracji bez poprawy
    liczba iteracji: 2588
    liczba iteracji z poprawą: 1588
    Czas obliczeń: 5343.81 sekund
    wynik - suma odległości: 178462.82859203394
    kolejność miast: [76 75  1 53 52 51 66 65 64 63 60 41 38 18 17 16 15 13 12  9 10  5  4  3
  6  7  8  2 22 24 45 48 44 25 21 59 61 57 56 55 42 43 28 29 30 32 33 54
 49 50 58 40 34 31 19 20 26 27 35 36 37 11 14 74 39 62 73 72 71 70 67 68
 69 47 46 23]
    Funkcja redukcji temperatury: szybkie wyżarzanie - temp = temp0/(iteracje+1)
    temperatura początkowa: 10000
    temperatura końcowa 3.8639876352395675
    Wylosowane miasta początkowe: [76, 4, 70, 53, 52, 47, 72, 7, 67, 42, 36, 40, 14, 29, 17, 49, 18, 8, 66, 12, 35, 28, 6, 46, 19, 9, 11, 1, 25, 69, 48, 2, 24, 38, 22, 30, 73, 57, 56, 15, 54, 64, 13, 10, 27, 63, 33, 44, 55, 50, 58, 75, 16, 20, 3, 59, 26, 23, 34, 21, 62, 37, 39, 74, 5, 43, 41, 65, 32, 71, 60, 68, 45, 51, 31, 61]
    Dały wartość początkową (pierwsza suma odległości

In [44]:
wyzarzanie_podsumowanie(
    TSP_76,
    kryterium_stopu=KryteriumStopu.iteracje,
    redukcja=RedukcjaTemperatury.stopniowa,
    temperatura_poczatkowa=2000,
    max_iteracje=4000
)

KeyboardInterrupt: 